Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# From 3D Segmentation to USD: A Complete Workflow for Hierarchical Mesh Conversion, USD Export, and NVIDIA Omniverse Integration
In this tutorial, we’ll cover:

- **Download the MONAI Bundle**: First, we download a pre-trained model bundle from the MONAI Model Zoo. This model bundle contains the necessary models and configurations for medical image analysis, which can accelerate our development process.

- **Run the Inference Workflow of the Bundle**: Using the downloaded model bundle, we run its built-in inference workflow to automatically segment or analyze the input medical imaging data and obtain the desired results.

- **Convert NIfTI/DICOM to Mesh**: The inference results are usually in NIfTI or DICOM format. We need to convert this volumetric data into a 3D mesh model for visualization and further processing.

- **Save the Mesh as OBJ and GLTF Formats**:
  - Save Single Mesh as OBJ Format: For a single organ or structure mesh, we save it in OBJ format, which is convenient to open and view in various 3D software.
  - Save Combined Mesh as GLTF Format: When we have meshes of multiple organs, we save them in GLTF format. This format preserves the hierarchical structure of the organs, making it easier to reflect the relationships between different organs during visualization.

- **Visualization in the Omniverse**: Finally, we import the GLTF format mesh into NVIDIA Omniverse. In Omniverse, we can utilize its powerful rendering and interactive capabilities to perform high-quality 3D visualization of medical imaging data, exploring the structures and spatial relationships of organs.

This end-to-end process enables efficient, high-quality visualization in NVIDIA Omniverse from raw segmentation data.


References:

[1] https://developer.nvidia.com/blog/advancing-surgical-robotics-with-ai-driven-simulation-and-digital-twin-technology/

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import skimage" || pip install -q scikit-image
!python -c "import vtk" || pip install -q vtk
!python -c "import pxr" || pip install -q usd-core
!python -c "import trimesh" || pip install -q trimesh
!python -c "import ipyvtklink" || pip install -q ipyvtklink
!apt update
!apt install -y libgl1-mesa-glx
!apt install libxrender1

## Setup imports

In [1]:
import os
import tempfile
import numpy as np

import vtk
import vtkmodules

# from ipyvtklink.viewer import ViewInteractiveWidget

from utility import convert_to_mesh, convert_mesh_to_usd

from monai.config import print_config
from monai.bundle.scripts import create_workflow, download
from monai.transforms import LoadImaged, SaveImage, Compose, BorderPadd, SqueezeDimd

print_config()

MONAI version: 1.4.1rc1
Numpy version: 1.24.4
Pytorch version: 2.5.0a0+872d972e41.nv24.08
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: e604d1841fe60c0ffb6978ae4116535ca8d8f34f
MONAI __file__: /workspace/Code/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.0
Pillow version: 10.4.0
Tensorboard version: 2.16.2
gdown version: 5.2.0
TorchVision version: 0.20.0a0
tqdm version: 4.66.5
lmdb version: 1.5.1
psutil version: 6.0.0
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.40.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/Data


## Generate segmentation from MAISI

### Download the MONAI Bundle
In this section, we download the MAISI bundle from monai model-zoo.

In [3]:
download(name="maisi_ct_generative", bundle_dir=root_dir)

2024-12-11 11:31:31,904 - INFO - --- input summary of monai.bundle.scripts.download ---
2024-12-11 11:31:31,905 - INFO - > name: 'maisi_ct_generative'
2024-12-11 11:31:31,905 - INFO - > bundle_dir: '/workspace/Data'
2024-12-11 11:31:31,905 - INFO - > source: 'monaihosting'
2024-12-11 11:31:31,906 - INFO - > remove_prefix: 'monai_'
2024-12-11 11:31:31,906 - INFO - > progress: True
2024-12-11 11:31:31,906 - INFO - ---


2024-12-11 11:31:32,611 - INFO - Expected md5 is None, skip md5 check for file /workspace/Data/maisi_ct_generative_v0.4.5.zip.
2024-12-11 11:31:32,612 - INFO - File exists: /workspace/Data/maisi_ct_generative_v0.4.5.zip, skipped downloading.
2024-12-11 11:31:32,612 - INFO - Writing into directory: /workspace/Data.


### Run the Inference Workflow of the Bundle
We use the `create_workflow` API from MONAI to streamline the inference process directly from the bundle.

Key input details for inference, such as the body region and target anatomy, are specified in [./configs/inference.json]. For a comprehensive explanation of the parameters, refer to [./docs/README.md] in the bundle directory. Additionally, we adjust the `output_size`, `spacing` and `num_splits` parameters to prevent out-of-memory issues during inference.

In [4]:
bundle_root = os.path.join(root_dir, "maisi_ct_generative")
override = {
    "output_size_xy": 256,
    "output_size_z": 256,
    "spacing_xy": 1.5,
    "spacing_z": 1.5,
    "autoencoder_def#num_splits": 16,
    "mask_generation_autoencoder_def#num_splits": 16,
}
workflow = create_workflow(
    config_file=os.path.join(bundle_root, "configs/inference.json"),
    workflow_type="inference",
    bundle_root=bundle_root,
    **override,
)

# uncomment this line to run the inference workflow
# then you will get the generated CT images and paired masks which can be used for the following steps.
# In this tutorial, we just use the tested data (IntegrationTest-AbdomenCT.nii.gz) from bundle for demonstration.
# workflow.run()

2024-12-11 11:32:14,010 - INFO - Setting logging properties based on config: /workspace/Data/maisi_ct_generative/configs/logging.conf.
2024-12-11 11:32:14,011 - py.warnings - WARNING - Detected deprecated name 'optional_packages_version' in configuration file, replacing with 'required_packages_version'.

2024-12-11 11:32:14,021 - INFO - --- input summary of monai.bundle.scripts.run ---
2024-12-11 11:32:14,022 - INFO - > workflow_type: 'inference'
2024-12-11 11:32:14,022 - INFO - > bundle_root: '/workspace/Data/maisi_ct_generative'
2024-12-11 11:32:14,022 - INFO - > output_size_xy: 256
2024-12-11 11:32:14,022 - INFO - > output_size_z: 256
2024-12-11 11:32:14,022 - INFO - > spacing_xy: 1.5
2024-12-11 11:32:14,023 - INFO - > spacing_z: 1.5
2024-12-11 11:32:14,023 - INFO - > autoencoder_def#num_splits: 16
2024-12-11 11:32:14,023 - INFO - > mask_generation_autoencoder_def#num_splits: 16
2024-12-11 11:32:14,023 - INFO - ---




## Convert NIfTI/DICOM to Mesh and Save as OBJ/GLTF

In this section, we convert the generated NII segmentation files into mesh format and save them as OBJ files. This process involves mapping labels to organs, exporting each organ as an individual mesh file, and generating a combined mesh file for all organs.

We define a function `nii_to_mesh` to handle the conversion of NIfTI files to OBJ files. The workflow is as follows:

- **Preprocessing**:
The function uses a series of transformations (`LoadImaged`, `BorderPadd`, and `SqueezeDimd`) to load and preprocess the input NIfTI file, ensuring it is ready for convertion.

- **Organ Label Mapping**:
It iterates over a dictionary mapping organ names to their respective label values. For each organ:
A binary mask (single_organ) is created to isolate the organ by assigning its corresponding label value.
The segmented organ is saved as a NIfTI file.

- **Mesh Conversion**:
Each segmented NIfTI file is converted into an OBJ file using the `convert_to_mesh` function.

- **Combined Mesh**:
A combined segmentation file is created by merging all organ segmentations into a single NIfTI file. This file is then converted into a GLTF file, preserving the hierarchical structure of the organs.


In [ ]:
# 17 groupings that cover 101 segments/regions out of 140
labels = {
    "Liver": 1,
    "Spleen": 3,
    "Pancreas": 4,
    "Heart": 115,
    "Body": 200,
    "Gallbladder": 10,
    "Stomach": 12,
    "Small_bowel": 19,
    "Colon": 62,
    "Kidney": {"right_kidney": 5, "left_kidney": 14},
    "Veins": {
        "aorta": 6,
        "inferior_vena_cava": 7,
        "portal_vein_and_splenic_vein": 17,
        "left_iliac_artery": 58,
        "right_iliac_artery": 59,
        "left_iliac_vena": 60,
        "right_iliac_vena": 61,
        "pulmonary_vein": 119,
        "left_subclavian_artery": 123,
        "right_subclavian_artery": 124,
        "superior_vena_cava": 125,
        "brachiocephalic_trunk": 109,
        "left_brachiocephalic_vein": 110,
        "right_brachiocephalic_vein": 111,
        "left_common_carotid_artery": 112,
        "right_common_carotid_artery": 113,
    },
    "Lungs": {
        "left_lung_upper_lobe": 28,
        "left_lung_lower_lobe": 29,
        "right_lung_upper_lobe": 30,
        "right_lung_middle_lobe": 31,
        "right_lung_lower_lobe": 32,
    },
    "Spine": {
        "vertebrae_L6": 131,
        "vertebrae_L5": 33,
        "vertebrae_L4": 34,
        "vertebrae_L3": 35,
        "vertebrae_L2": 36,
        "vertebrae_L1": 37,
        "vertebrae_T12": 38,
        "vertebrae_T11": 39,
        "vertebrae_T10": 40,
        "vertebrae_T9": 41,
        "vertebrae_T8": 42,
        "vertebrae_T7": 43,
        "vertebrae_T6": 44,
        "vertebrae_T5": 45,
        "vertebrae_T4": 46,
        "vertebrae_T3": 47,
        "vertebrae_T2": 48,
        "vertebrae_T1": 49,
        "vertebrae_C7": 50,
        "vertebrae_C6": 51,
        "vertebrae_C5": 52,
        "vertebrae_C4": 53,
        "vertebrae_C3": 54,
        "vertebrae_C2": 55,
        "vertebrae_C1": 56,
        "sacrum": 97,
        "vertebrae_S1": 127,
    },
    "Ribs": {
        "left_rib_1": 63,
        "left_rib_2": 64,
        "left_rib_3": 65,
        "left_rib_4": 66,
        "left_rib_5": 67,
        "left_rib_6": 68,
        "left_rib_7": 69,
        "left_rib_8": 70,
        "left_rib_9": 71,
        "left_rib_10": 72,
        "left_rib_11": 73,
        "left_rib_12": 74,
        "right_rib_1": 75,
        "right_rib_2": 76,
        "right_rib_3": 77,
        "right_rib_4": 78,
        "right_rib_5": 79,
        "right_rib_6": 80,
        "right_rib_7": 81,
        "right_rib_8": 82,
        "right_rib_9": 83,
        "right_rib_10": 84,
        "right_rib_11": 85,
        "right_rib_12": 86,
        "costal_cartilages": 114,
        "sternum": 122,
    },
    "Shoulders": {"left_scapula": 89, "right_scapula": 90, "left_clavicula": 91, "right_clavicula": 92},
    "Hips": {"left_hip": 95, "right_hip": 96},
    "Back_muscles": {
        "left_gluteus_maximus": 98,
        "right_gluteus_maximus": 99,
        "left_gluteus_medius": 100,
        "right_gluteus_medius": 101,
        "left_gluteus_minimus": 102,
        "right_gluteus_minimus": 103,
        "left_autochthon": 104,
        "right_autochthon": 105,
        "left_iliopsoas": 106,
        "right_iliopsoas": 107,
    },
}


def nii_to_mesh(input_nii_path, output_nii_path, output_obj_path):
    """
    This function converts each organ into a separate OBJ file and generates a GLTF file
    containing all organs with hierarchical structure.
    It processes the input NIfTI file and groups 140 labels into 17 categories.

    Args:
        input_nii_path: path to the nii file
        output_nii_path: path to save the obj files
        output_obj_path: path to save the gltf file
    """
    if not os.path.exists(output_nii_path):
        os.makedirs(output_nii_path)
    pre_trans = Compose(
        [
            LoadImaged(keys="label", ensure_channel_first=True),
            BorderPadd(keys="label", spatial_border=2),
            SqueezeDimd(keys="label", dim=0),
        ]
    )
    orig_seg = pre_trans({"label": input_nii_path})["label"]
    all_organ = np.zeros_like(orig_seg, dtype=np.uint8)
    all_label_values = {}

    save_trans = SaveImage(output_ext="nii.gz", output_dtype=np.uint8)
    for j, (organ_name, label_val) in enumerate(labels.items(), start=1):
        single_organ = np.zeros_like(orig_seg, dtype=np.uint8)
        print(f"Assigning index {j} to label {organ_name}")
        if isinstance(label_val, dict):
            for _, i in label_val.items():
                all_organ[orig_seg == i] = j
                single_organ[orig_seg == i] = j
        else:
            all_organ[orig_seg == label_val] = j
            single_organ[orig_seg == label_val] = j
        organ_filename = os.path.join(output_nii_path, organ_name)
        save_trans(single_organ[None], meta_data=orig_seg.meta, filename=organ_filename)
        convert_to_mesh(
            f"{organ_filename}.nii.gz",
            output_obj_path,
            f"{organ_name}.obj",
            label_value=j,
            smoothing_factor=0.5,
            reduction_ratio=0.0,
        )
        all_label_values[j] = organ_name

    all_organ_filename = os.path.join(output_nii_path, "all_organs")
    save_trans(all_organ[None], meta_data=orig_seg.meta, filename=all_organ_filename)
    convert_to_mesh(
        f"{all_organ_filename}.nii.gz",
        output_obj_path,
        "all_organs.gltf",
        label_value=all_label_values,
        smoothing_factor=0.6,
        reduction_ratio=0.0,
    )
    print(f"Saved whole segmentation {all_organ_filename}")


# if workflow.run() is uncommented and run, an image and a label files will be generated in the output folder
# for example, file paths are like this:
# f"{bundle_root}/output/sample_20250317_065034_415537_image.nii.gz"
# f"{bundle_root}/output/sample_20250317_065034_415537_label.nii.gz"
# then you can replace the "input_nii_path" below with the generated label file
input_nii_path = f"{bundle_root}/datasets/IntegrationTest-AbdomenCT.nii.gz"
output_nii_path = f"{bundle_root}/datasets/monai/nii"
output_obj_path = f"{bundle_root}/datasets/monai/obj"
out = nii_to_mesh(input_nii_path, output_nii_path, output_obj_path)

## Convert 3D model contain all organs to USD format

[Universal Scene Description (OpenUSD)](https://openusd.org/release/index.html) is an extensible ecosystem of file formats, compositors, renderers, and other plugins for comprehensive 3D scene description.

Instead of using code to convert, you can also use the built-in converter in NVIDIA Omniverse to perform the conversion. For more details on using the CAD Converter extension, please refer to the official documentation:
https://docs.omniverse.nvidia.com/extensions/latest/ext_cad-converter/manual.html

In [4]:
obj_filename = f"{bundle_root}/datasets/monai/obj/all_organs_modified.gltf"
usd_filename = f"{bundle_root}/datasets/monai/obj/all_organs.usd"

convert_mesh_to_usd(obj_filename, usd_filename)

USD file successfully exported to /workspace/Data/maisi_ct_generative/datasets/monai/obj/all_organs.usd


## Visualize one single organ mesh

Here we randomly select one organ to visualize the mesh using `ViewInteractiveWidget`

In [ ]:
# Step 1: Read the mesh
reader = vtk.vtkOBJReader()
reader.SetFileName(f"{bundle_root}/datasets/monai/obj/Spleen.obj")
reader.Update()

# Step 2: Create a mapper
mapper = vtkmodules.vtkRenderingCore.vtkPolyDataMapper()
mapper.SetInputData(reader.GetOutput())
print(f"Number of Points: {mapper.GetInput().GetNumberOfPoints()}")
print(f"Number of Cells: {mapper.GetInput().GetNumberOfCells()}")

# Step 3: Create an actor
actor = vtkmodules.vtkRenderingCore.vtkActor()
actor.SetMapper(mapper)

# Step 4: Create a renderer
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.SetSize(800, 600)
render_window.SetOffScreenRendering(1)

# Step 5: Create a render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)
interactor_style = vtk.vtkInteractorStyleTrackballCamera()
render_window_interactor.SetInteractorStyle(interactor_style)

# Uncomment the following line to display the interactive widget
# render_window.Render()
# interactive_widget = ViewInteractiveWidget(render_window)
# interactive_widget

![Spleen](Spleen.png)

## Visualization in the Omniverse

Since the Omniverse Launcher has been deprecated, you can use USD Composer to view and manipulate the OpenUSD output files.

To download the latest version of USD Composer, you can follow the instructions here to build and launch:

https://github.com/NVIDIA-Omniverse/kit-app-template/tree/main/templates/apps/usd_composer#getting-started

![omniverse](./omniverse.png)

## Visualization of All Organs

You can view the 3D models using online viewer such as https://3dviewer.net/#

![all organs](result.png)